# UML

In [ ]:
import iplantuml

### State Machine UML

In [ ]:
%%plantuml

@startuml

state Unlocked {
    
    [*] -> Disconnected
    
    Unlocked : <b>pbNewContact</b> enabled <b>true</b>
    Unlocked : <b>pbSend</b> enabled <b>true</b>
    Unlocked : <b>pbAttachFile</b> enabled <b>true</b>
    Unlocked : <b>lwContacts</b> enabled <b>true</b>
    Unlocked : <b>teSend</b> enabled <b>true</b>
    Unlocked : <b>lbAttachFile</b> enabled <b>true</b>
    Unlocked : <b>pbDeleteContact</b> enabled <b>false</b>
    Unlocked : <b>pbEditContact</b> enabled <b>false</b>
    
    state Disconnected {
    
        Disconnected : <b>pbNewContact</b> enabled <b>true</b>
        Disconnected : <b>pbSend</b> enabled <b>false</b>
        Disconnected : <b>pbAttachFile</b> enabled <b>false</b>
        Disconnected : <b>lwContacts</b> enabled <b>true</b>
        Disconnected : <b>teSend</b> enabled <b>false</b>
        Disconnected : <b>lbAttachFile</b> enabled <b>false</b>
        Disconnected : <b>pbDeleteContact</b> enabled <b>false</b>
        Disconnected : <b>pbEditContact</b> enabled <b>false</b>
    }
    
    state Connected {
        
        Connected : <b>pbNewContact</b> enabled <b>true</b>
        Connected : <b>pbSend</b> enabled <b>false</b>
        Connected : <b>pbAttachFile</b> enabled <b>true</b>
        Connected : <b>lwContacts</b> enabled <b>true</b>
        Connected : <b>teSend</b> enabled <b>true</b>
        Connected : <b>lbAttachFile</b> enabled <b>true</b>
        Connected : <b>pbDeleteContact</b> enabled <b>false</b>
        Connected : <b>pbEditContact</b> enabled <b>false</b>
    }
    
    state ValidateSendable {
    
        ValidateSendable: <b>state</b> entered() -> <b>this</b> on_validateSendable()
    }
    
    state Sendable {
    
        Sendable : <b>pbNewContact</b> enabled <b>true</b>
        Sendable : <b>pbSend</b> enabled <b>true</b>
        Sendable : <b>pbAttachFile</b> enabled <b>true</b>
        Sendable : <b>lwContacts</b> enabled <b>true</b>
        Sendable : <b>teSend</b> enabled <b>true</b>
        Sendable : <b>lbAttachFile</b> enabled <b>true</b>
        Sendable : <b>pbDeleteContact</b> enabled <b>false</b>
        Sendable : <b>pbEditContact</b> enabled <b>false</b>
    }
    
    note top of ValidateSendable
        - checks if message is non empty (contains text and/or file)
        - unlocks pbSend only if above is true
    end note
    
    Locked -up-> [H] : <b>this</b> contactAdded()
    Locked -up-> [H] : <b>this</b> contactAdditionCanceled()
    Locked -up-> [H] : <b>this</b> errorCatched()
    
    Disconnected -down-> Connected : <b>this</b> contactConnected()
    
    Connected -up-> Disconnected : <b>this</b> contactDisconnected()
    Connected -right-> ValidateSendable : <b>teSend</b> textChanged()
    Connected -right-> ValidateSendable : <b>this</b> fileChanged()
    
    ValidateSendable -down-> Sendable : <b>this</b> msgSendable()
    ValidateSendable -left-> Connected : <b>this</b> msgUnsendable()
    
    Sendable -up-> ValidateSendable : <b>this</b> fileChanged()
    Sendable -up-> ValidateSendable : <b>teSend</b> textChanged()
}

state Locked {

    Locked : <b>pbNewContact</b> enabled <b>false</b>
    Locked : <b>pbSend</b> enabled <b>false</b>
    Locked : <b>pbAttachFile</b> enabled <b>false</b>
    Locked : <b>lwContacts</b> enabled <b>false</b>
    Locked : <b>teSend</b> enabled <b>false</b>
    Locked : <b>lbAttachFile</b> enabled <b>false</b>
    Locked : <b>pbDeleteContact</b> enabled <b>false</b>
    Locked : <b>pbEditContact</b> enabled <b>false</b>
}

[*] --> Unlocked
Unlocked -down-> Locked : <b>pbNewContact</b> clicked()
Unlocked -down-> Locked : <b>this</b> error(QString)


@enduml